In [3]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px

In [4]:
mt5.initialize(login=1700523, server='GenialInvestimentos-PRD',
               password='C21j17+eh')
pd.set_option('display.max_rows', None)

In [5]:
symbol = 'WODK24'
timeframe = mt5.TIMEFRAME_M5
start_pos = 0
num_bars = 1000
volume = 1

In [188]:
import pandas as pd
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()

rates = mt5.copy_rates_from_pos("WDOK24", mt5.TIMEFRAME_M5, 0, 2000)


mt5.shutdown()


rates_frame = pd.DataFrame(rates)
rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')


rates_frame['time'] = pd.to_datetime(rates_frame['time'])
rates_frame['date'] = rates_frame['time'].dt.date

closing_prices = rates_frame.groupby('date')['close'].last()
previous_closing_prices = closing_prices.shift(1)

date_to_previous_close = previous_closing_prices.to_dict()

rates_frame['previous_day_close'] = rates_frame['date'].map(
    date_to_previous_close)
daily_close = rates_frame.groupby('date')['close'].last()

rates_frame['previous_close'] = rates_frame['date'].map(daily_close.shift(1))

rates_frame.dropna(subset=['previous_close'], inplace=True)

rates_frame['1pct_change_up'] = rates_frame['previous_day_close'] * (1 + 0.01)
rates_frame['1pct_change_down'] = rates_frame['previous_day_close'] * \
    (1 - 0.01)

rates_frame.head()

,time,open,high,low,close,tick_volume,spread,real_volume,date,previous_day_close,previous_close,1pct_change_up,1pct_change_down
62,2024-04-08 09:00:00,5082.0,5086.5,5077.5,5080.0,18301,1,58457,2024-04-08,5080.0,5080.0,5130.8,5029.2
63,2024-04-08 09:05:00,5080.0,5081.0,5074.5,5075.5,9284,1,31017,2024-04-08,5080.0,5080.0,5130.8,5029.2
64,2024-04-08 09:10:00,5075.5,5077.5,5074.0,5075.5,6790,1,23331,2024-04-08,5080.0,5080.0,5130.8,5029.2
65,2024-04-08 09:15:00,5076.0,5078.0,5074.5,5078.0,5068,1,17696,2024-04-08,5080.0,5080.0,5130.8,5029.2
66,2024-04-08 09:20:00,5078.0,5082.0,5075.5,5076.5,12390,1,37178,2024-04-08,5080.0,5080.0,5130.8,5029.2


In [189]:
fig = go.Figure(data=[go.Candlestick(
    x=rates_frame['time'].dt.strftime('%d/%m %H:%M'),
    open=rates_frame['open'],
    high=rates_frame['high'],
    low=rates_frame['low'],
    close=rates_frame['close']
)])

fig.update_xaxes(type='category')
fig.update_layout(
    xaxis=dict(
        tickmode='auto',
        nticks=10
    )
)

times = rates_frame['time'].dt.strftime('%d/%m %H:%M').tolist()
previous_closes = rates_frame['previous_day_close'].tolist()
one_pct_ups = rates_frame['1pct_change_up'].tolist()
one_pct_downs = rates_frame['1pct_change_down'].tolist()


fig.add_trace(go.Scatter(
    x=times,
    y=previous_closes,
    mode='markers',
    marker=dict(color='RoyalBlue', size=3),
    name='Previous Day Close'
))

fig.add_trace(go.Scatter(
    x=times,
    y=one_pct_ups,
    mode='markers',
    marker=dict(color='Red', size=3),
    name='1% Change Up'
))

fig.add_trace(go.Scatter(
    x=times,
    y=one_pct_downs,
    mode='markers',
    marker=dict(color='Red', size=3),
    name='1% Change Down'
))

fig.update_xaxes(type='category')
fig.update_layout(
    title='Stock Price Analysis',
    xaxis_title='Date and Time',
    yaxis_title='Price',
    legend_title='Indicator',
    xaxis=dict(
        tickmode='auto',
        nticks=10
    )
)

fig.show()

In [200]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, one_pct_change_up, one_pct_change_down, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.one_pct_change_up = one_pct_change_up
        self.one_pct_change_down = one_pct_change_down
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'one_pct_change_up': self.one_pct_change_up,
            'one_pct_change_down': self.one_pct_change_down,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.data = df
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []

    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = (df['profit'].cumsum() * 10) + self.starting_balance
        return df

    def add_position(self, position):
        self.positions.append(position)

    def run(self):
        take_profit_points = 10
        stop_loss_points = 10

        for i, row in self.data.iterrows():
            if i == 0:
                continue

            print(row['date'])
            if i < len(self.data) - 1:
                next_row = self.data.iloc[i + 1]
                is_last_candle_of_day = row['date'] != next_row['date']
            else:
                is_last_candle_of_day = True

            open_positions = [p for p in self.positions if p.status == 'open']

            if not open_positions:
                if row['close'] >= row['1pct_change_up']:
                    sl = row['close'] + take_profit_points
                    tp = row['close'] - stop_loss_points
                    self.add_position(Position(row['time'], row['close'], row['1pct_change_up'], row['1pct_change_down'], 'sell', self.volume, sl, tp))
                elif row['close'] <= row['1pct_change_down']:
                    sl = row['close'] - take_profit_points
                    tp = row['close'] + stop_loss_points
                    self.add_position(Position(row['time'], row['close'], row['1pct_change_up'], row['1pct_change_down'], 'buy', self.volume, sl, tp))

            for position in open_positions:
                if (position.order_type == 'sell' and row['close'] <= position.tp) or \
                   (position.order_type == 'buy' and row['close'] >= position.tp) or \
                   (position.order_type == 'sell' and row['close'] >= position.sl) or \
                   (position.order_type == 'buy' and row['close'] <= position.sl):
                    position.close_position(row['time'], row['close'])

            if row['time'].strftime('%H:%M:%S') == '18:25:00':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(row['time'], row['close'])
                        # position.status = 'closed'
                continue


        return self.get_positions_df()



In [201]:
df = rates_frame.copy()
starting_balance = 1000
volume = 1
# rates_frame
strategy = Strategy(df, starting_balance, volume)

positions_df = strategy.run()
positions_df

2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08
2024-04-08

,open_datetime,one_pct_change_up,one_pct_change_down,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2024-04-10 10:10:00,5066.665,4966.335,5069.0,sell,1,5079.0,5059.0,2024-04-10 12:00:00,5081.0,-12.0,closed,880.0
1,2024-04-10 12:05:00,5066.665,4966.335,5082.5,sell,1,5092.5,5072.5,2024-04-10 14:40:00,5094.0,-11.5,closed,765.0
2,2024-04-10 14:45:00,5066.665,4966.335,5088.5,sell,1,5098.5,5078.5,2024-04-10 18:25:00,5078.0,10.5,closed,870.0
3,2024-04-12 11:55:00,5153.020,5050.980,5154.5,sell,1,5164.5,5144.5,2024-04-12 13:00:00,5142.5,12.0,closed,990.0
4,2024-04-15 10:20:00,5178.775,5076.225,5179.0,sell,1,5189.0,5169.0,2024-04-15 10:30:00,5194.5,-15.5,closed,835.0
5,2024-04-15 10:35:00,5178.775,5076.225,5188.0,sell,1,5198.0,5178.0,2024-04-15 14:20:00,5205.5,-17.5,closed,660.0
6,2024-04-15 14:25:00,5178.775,5076.225,5212.0,sell,1,5222.0,5202.0,2024-04-15 14:50:00,5199.0,13.0,closed,790.0
7,2024-04-15 14:55:00,5178.775,5076.225,5195.0,sell,1,5205.0,5185.0,2024-04-15 18:25:00,5193.5,1.5,closed,805.0
8,2024-04-16 09:25:00,5245.435,5141.565,5255.0,sell,1,5265.0,5245.0,2024-04-16 09:30:00,5242.5,12.5,closed,930.0
9,2024-04-16 09:50:00,5245.435,5141.565,5246.0,sell,1,5256.0,5236.0,2024-04-16 10:15:00,5256.5,-10.5,closed,825.0


In [206]:
num_profitable_trades = (positions_df['profit'] > 0).sum()


num_loss_trades = (positions_df['profit'] < 0).sum()

total_trades = len(positions_df)
profitability_ratio = num_profitable_trades / total_trades if total_trades > 0 else 0
average_profit = positions_df.loc[positions_df['profit'] > 0, 'profit'].mean()
average_loss = positions_df.loc[positions_df['profit'] < 0, 'profit'].mean()

cumulative_profit = positions_df['profit'].cumsum()
running_max = cumulative_profit.cummax()
drawdown = (running_max - cumulative_profit).max()

total_profit = positions_df['profit'].sum()
initial_balance = positions_df.iloc[0]['open_price']
return_percent = total_profit / initial_balance * 100

total_profit = positions_df.loc[positions_df['profit'] > 0, 'profit'].sum()
total_loss = positions_df.loc[positions_df['profit'] < 0, 'profit'].sum()

print(f"Total de operações: {total_trades}")
print(f"Operações lucrativas: {num_profitable_trades}")
print(f"Operações com prejuízo: {num_loss_trades}")
print(f"Eficácia da estratégia: {profitability_ratio:.2%}")

print('\n')
print(f"Total de lucro das operações lucrativas: ${total_profit:.2f}")
print(f"Total de prejuízo das operações com prejuízo: ${total_loss:.2f}")

print('\n')
print(f"Lucro médio: ${average_profit:.2f}")
print(f"Prejuízo médio: ${average_loss:.2f}")
print(f"Drawdown máximo: ${drawdown:.2f}")
print(f"Retorno total: ${total_profit:.2f}")
print(f"Retorno percentual: {return_percent:.2f}%")



Total de operações: 27
Operações lucrativas: 15
Operações com prejuízo: 12
Eficácia da estratégia: 55.56%


Total de lucro das operações lucrativas: $151.00
Total de prejuízo das operações com prejuízo: $-139.00


Lucro médio: $10.07
Prejuízo médio: $-11.58
Drawdown máximo: $47.50
Retorno total: $151.00
Retorno percentual: 0.24%


In [202]:
px.line(positions_df, x='close_datetime', y='pnl')

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [199]:
import plotly.graph_objects as go
import pandas as pd


fig = go.Figure(data=[go.Candlestick(
    x=rates_frame['time'].dt.strftime('%Y-%m-%d %H:%M'),
    open=rates_frame['open'],
    high=rates_frame['high'],
    low=rates_frame['low'],
    close=rates_frame['close']
)])

fig.add_trace(go.Scatter(
    x=rates_frame['time'].dt.strftime('%Y-%m-%d %H:%M'),
    y=rates_frame['1pct_change_up'],
    mode='markers',
    marker=dict(color='Grey', size=3),
    name='Entry Points'
))

fig.add_trace(go.Scatter(
    x=rates_frame['time'].dt.strftime('%Y-%m-%d %H:%M'),
    y=rates_frame['1pct_change_down'],
    mode='markers',
    marker=dict(color='Grey', size=3),
    name='Entry Points'
))

fig.add_trace(go.Scatter(
    x=positions_df['open_datetime'].dt.strftime('%Y-%m-%d %H:%M'),
    y=positions_df['open_price'],
    mode='markers',
    marker=dict(color='Green', size=3),
    name='Entry Points'
))

fig.add_trace(go.Scatter(
    x=positions_df['close_datetime'].dt.strftime('%Y-%m-%d %H:%M'),
    y=positions_df['close_price'],
    mode='markers',
    marker=dict(color='Red', size=3),
    name='Exit Points'
))

fig.update_xaxes(type='category')
fig.update_layout(
    title='Trading Operations Visualization',
    xaxis_title='Date and Time',
    yaxis_title='Price',
    legend_title='Indicator'
)

fig.show()

In [62]:
# visualize close price
fig_2 = px.line(df, x='time', y=['close'])

# for i, row in signal.iterrows():
#     fig.add_vline(x=row.time)

for i, row in positions_df[positions_df['status'] == 'closed'].iterrows():

    if row.profit > 0:
        fig.add_shape(type="line",
                      x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
                      line=dict(color="Green", width=3)
                      )

    elif row.profit < 0:
        fig.add_shape(type="line",
                      x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
                      line=dict(color="Red", width=3)
                      )


fig_2.show()

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

